<a href="https://colab.research.google.com/github/Leonardowso/Portfolio_power_bi/blob/main/MODELO_ANALISE_CARGA_API_3_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

#### 0  Carregar o arquivo CSV
df = pd.read_csv("Rotas.csv")



#### 1 Converter datas para formato correto DIA MES ANO (caso haja falhas no parse automático)


df['Dt.Pedido'] = pd.to_datetime(df['Dt.Pedido'], format='%d/%m/%y', errors='coerce')
df['Dt.Emissao'] = pd.to_datetime(df['Dt.Emissao'], format='%d/%m/%y', errors='coerce')
df['Dt.Entrega'] = pd.to_datetime(df['Dt.Entrega'], format='%d/%m/%y', errors='coerce')

# Converter datas para formato correto ANO MES DIA(caso haja falhas no parse automático)
df['Dt.Pedido'] = pd.to_datetime(df['Dt.Pedido'], format='%y/%m/%d', errors='coerce')
df['Dt.Emissao'] = pd.to_datetime(df['Dt.Emissao'], format='%y/%m/%d', errors='coerce')
df['Dt.Entrega'] = pd.to_datetime(df['Dt.Entrega'], format='%y/%m/%d', errors='coerce')



## Função para verificar se alguma das datas está em 2022
def contem_2022(row):
    return any(
        dt.year == 2022 if pd.notnull(dt) else False
        for dt in [row['Dt.Pedido'], row['Dt.Emissao'], row['Dt.Entrega']]
    )

# Apagar linhas que possuem qualquer data do ano de 2022
df = df[~df.apply(contem_2022, axis=1)]
# Corrigir emissões após a entrega (exemplo simplificado)
df['Dt.Emissao'] = df.apply(
    lambda row: row['Dt.Pedido'] if row['Dt.Emissao'] > row['Dt.Entrega'] else row['Dt.Emissao'],
    axis=1
)

# Remover linhas com datas inválidas
df = df.dropna(subset=['Dt.Pedido', 'Dt.Emissao', 'Dt.Entrega'])
parse_dates=['Dt.Pedido', 'Dt.Emissao', 'Dt.Entrega']




#### 2 Criar flags para inconsistências


df['Inconsistencia_Entrega_Antes_Pedido'] = df['Dt.Entrega'] < df['Dt.Pedido']
df['Inconsistencia_Emissao_Apos_Entrega'] = df['Dt.Emissao'] > df['Dt.Entrega']

# Filtrar registros problemáticos
inconsistencias = df[df['Inconsistencia_Entrega_Antes_Pedido'] | df['Inconsistencia_Emissao_Apos_Entrega']]
print(f"Registros inconsistentes: {len(inconsistencias)}")

# Mostra todas as linhas com inconsistências
print(inconsistencias)

# Remove as inconsistências do DataFrame original (sem criar um novo)
df = df[~(df['Inconsistencia_Entrega_Antes_Pedido'] | df['Inconsistencia_Emissao_Apos_Entrega'])].copy()


print(f"\nDataFrame após remoção: {len(df)} registros (inconsistências excluídas)")






#### 3 LEADTIME

import matplotlib.pyplot as plt

# Histograma de prazos de entrega

df['Prazo_Entrega'] = (df['Dt.Entrega'] - df['Dt.Pedido']).dt.days




#### 4  ANALISES DE VALORES (vlr.frete)



####  Converter a coluna "Vlr.Frete" para o formato 0000.00 (FLOAT para cálculo)
df["Vlr.Frete"] = df["Vlr.Frete"].astype(str).str.replace(',', '.', regex=False).astype(float)

#  Valor unitário por item transportado
df["Valor.Por.Unidade"] = df["Vlr.Frete"] / df["Qtd.Transp"]

# ✅ Formatar "Vlr.Frete" como string "0.000,00" (para exibição)
df["Vlr.Frete"] = df["Vlr.Frete"].apply(lambda x: f"{x:,.2f}".replace(',', 'TEMP').replace('.', ',').replace('TEMP', '.') if pd.notnull(x) else None)


# ✅ Formatar "Valor.Por.Unidade" como string "0,00" (para exibição)
df["Valor.Por.Unidade"] = df["Valor.Por.Unidade"].apply(lambda x: f"{x:.2f}".replace('.', ',') if pd.notnull(x) else None)

####### IDENTIFICAR AS LINHAS FOB>0 E ALTERAR PARA CIF ###############
### Substituir vírgula por ponto e remover pontos de milhar (formato brasileiro)
df['Vlr.Frete.Num'] = (
    df['Vlr.Frete']
    .astype(str)
    .str.replace('.', '', regex=False)    # Remove milhar
    .str.replace(',', '.', regex=False)   # Troca vírgula por ponto
)

# Converte para float (valores inválidos viram NaN)
df['Vlr.Frete.Num'] = pd.to_numeric(df['Vlr.Frete.Num'], errors='coerce')

# Filtrar registros onde Incoterm é FOB e valor de frete > 0
fob_com_frete = df[(df['Incoterm'] == 'FOB') & (df['Vlr.Frete.Num'] > 0)].copy()

# Mostrar resultado
print(f"\nRegistros FOB com Vlr.Frete > 0: {len(fob_com_frete)} encontrados\n")
display(fob_com_frete)

# Corrigir valor de frete (formato brasileiro) para float
df['Vlr.Frete.Num'] = (
    df['Vlr.Frete']
    .astype(str)
    .str.replace('.', '', regex=False)    # Remove milhar
    .str.replace(',', '.', regex=False)   # Troca vírgula por ponto decimal
)

df['Vlr.Frete.Num'] = pd.to_numeric(df['Vlr.Frete.Num'], errors='coerce')

# Criar condição
condicao_fob_com_frete = (df['Incoterm'] == 'FOB') & (df['Vlr.Frete.Num'] > 0)

# Aplicar a substituição
df.loc[condicao_fob_com_frete, 'Incoterm'] = 'CIF'

# (Opcional) Exibir quantas linhas foram alteradas
print(f"Linhas alteradas de FOB para CIF: {condicao_fob_com_frete.sum()}")





##### 5. Taxa de ocupação do veículo
# Cada pallet vale 150 unidades (P12 = 1800 unidades, P24 = 3600 unidades)

# Verificar se as cargas para P12 excedem 1800 unidades
p12_excesso = df[(df['Veiculo'] == 'P12') & (df['Qtd.Transp'] > 1800)]

print(f"Registros P12 com carga acima de 1800: {len(p12_excesso)}")
if not p12_excesso.empty:
    print("Registros P12 com excesso de carga:")
    display(p12_excesso[['Veiculo', 'Qtd.Transp']])

    # Alterar 'P12' para 'P24' para os registros com Qtd.Transp > 1800 (assumindo que 3600 é a Qtd.Transp em excesso)
    df.loc[p12_excesso.index, 'Veiculo'] = 'P24'
    print("\nRegistros com veículo atualizado para P24:")
    display(df.loc[p12_excesso.index, ['Veiculo', 'Qtd.Transp']])


# Verificar se as cargas para P24 excedem 3600 unidades
p24_excesso = df[(df['Veiculo'] == 'P24') & (df['Qtd.Transp'] > 3600)]

print(f"\nRegistros P24 com carga acima de 3600: {len(p24_excesso)}")
if not p24_excesso.empty:
    print("Registros P24 com excesso de carga:")
    display(p24_excesso[['Veiculo', 'Qtd.Transp']])

# Verificar se existem outros tipos de veículo na coluna "Veiculo"
outros_veiculos = df[~df['Veiculo'].isin(['P12', 'P24'])]

print(f"\nRegistros com outros tipos de veículo: {len(outros_veiculos)}")
if not outros_veiculos.empty:
    print("Outros tipos de veículo encontrados:")
    display(outros_veiculos['Veiculo'].value_counts())
    print("\nRegistros com outros tipos de veículo:")
    display(outros_veiculos[['Veiculo', 'Qtd.Transp']])


### pallets por pedidos otimização(BASTOS)

df['Pallets_Transportados'] = df['Qtd.Transp'] / 150

# ✅ Formatar "Pallets_Transportados" como string "00" (para exibição)

df['Pallets_Transportados'] = df['Pallets_Transportados'].apply(lambda x: f"{x:.0f}" if pd.notnull(x) else None)
print(df['Pallets_Transportados'])


## TAXA DE OCUPAÇÃO

import numpy as np

df["Taxa.Ocupacao(%)"] = np.where(
    df["Veiculo"] == "P12",
    round((df["Qtd.Transp"] / 1800) * 100, 2),
    np.where(
        df["Veiculo"] == "P24",
        round((df["Qtd.Transp"] / 3600) * 100, 2),
        None
    )
)

# ✅ Formatar como porcentagem com duas casas decimais (para exibição)
df["Taxa.Ocupacao(%)"] = df["Taxa.Ocupacao(%)"].apply(
    lambda x: f"{x:.2f}%" if pd.notnull(x) else None
)




###################################################################################
################# CAPACIDADE DE PRODUÇÃO ANUAL DAS 3 FABRICAS #####################


#Fábrica        Capacidade (CIF)
#3403208_17000000_UND
#3423909_12500000_UND
#3424402_19500000_UND




###############################################################################
################# FABRICAS E SEUS RESPECITIVOS CLIENTES  ######################






# Agrupar e obter clientes únicos
clientes_por_fabrica = df.groupby('CO.Fabrica')['CO.Cliente'].unique().reset_index()

# Transformar listas de clientes em strings separadas por vírgula
clientes_por_fabrica['CO.Cliente'] = clientes_por_fabrica['CO.Cliente'].apply(lambda x: ','.join(map(str, x)))

# Salvar como CSV separado por vírgula
clientes_por_fabrica.to_csv('clientes_por_fabrica.csv', index=False)







###################################################################################
###################### (CIF) DEMANDA ANUAL DE CARGA PARA CADA CLIENTE##############





# Garantir tipo numérico
df['Qtd.Transp'] = pd.to_numeric(df['Qtd.Transp'], errors='coerce')

# Filtrar apenas linhas com Incoterm = CIF
df_cif = df[df['Incoterm'] == 'CIF'].copy()

# Agrupar por cliente e somar
soma_cif_por_cliente = df_cif.groupby('CO.Cliente')['Qtd.Transp'].sum().reset_index()

# Renomear coluna para clareza
soma_cif_por_cliente.rename(columns={'Qtd.Transp': 'Soma.Qtd.CIF'}, inplace=True)

# Exibir resultado
print("Soma da quantidade transportada por cliente (somente CIF):")
print(soma_cif_por_cliente)

# TABELA SOMA QTD TRANSPORTADA POR CLIENTE

soma_cif_por_cliente.to_csv('soma_qtd_transportada_CIF_por_cliente.csv', index=False, encoding='utf-8-sig')

print("\nArquivo 'soma_qtd_transportada_CIF_por_cliente.csv' salvo com sucesso!")

soma_cif_por_cliente.to_csv('soma_qtd_transportada_CIF_por_cliente.csv', index=False, encoding='utf-8-sig')

print("\nArquivo 'soma_qtd_transportada_CIF_por_cliente.csv' salvo com sucesso!")







################################################################################
################ (CIF) MEDIA DE CUSTO POR UNIDADE TRANSPORTADA #################


# Corrigir valor de frete (formato brasileiro) para float
df['Vlr.Frete.Num'] = (
    df['Vlr.Frete']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)
df['Vlr.Frete.Num'] = pd.to_numeric(df['Vlr.Frete.Num'], errors='coerce')
df['Qtd.Transp'] = pd.to_numeric(df['Qtd.Transp'], errors='coerce')

# Filtrar somente linhas com Incoterm = CIF
df_cif = df[(df['Incoterm'] == 'CIF') & (df['Qtd.Transp'] > 0) & df['Vlr.Frete.Num'].notnull()].copy()

# Calcular valor unitário por linha
df_cif['Vlr.Unitario'] = df_cif['Vlr.Frete.Num'] / df_cif['Qtd.Transp']

# Calcular média por cliente
media_por_cliente = df_cif.groupby('CO.Cliente')['Vlr.Unitario'].mean().reset_index()

# Formatar a média no padrão 0,00 (brasileiro)
media_por_cliente['Vlr.Unitario'] = media_por_cliente['Vlr.Unitario'].apply(
    lambda x: f"{x:,.2f}".replace('.', 'v').replace(',', '.').replace('v', ',')
)

# Salvar resultado em CSV
media_por_cliente.to_csv('media_valor_unitario_CIF_por_cliente.csv', index=False, encoding='utf-8-sig')

print("\n✅ Arquivo 'media_valor_unitario_CIF_por_cliente.csv' salvo com sucesso!")



########################### OTIMIZAÇÃO CIF ############################################




!pip install pulp
from pulp import *
import pandas as pd

# 1. Capacidade das fábricas
fabricas = {
    '3403208': 17000000,
    '3423909': 12500000,
    '3424402': 19500000
}

# 2. Demandas dos clientes
demandas = {
    '2301': 7800, '2302': 5400, '2303': 5400, '2304': 404700, '2305': 601200,
    '2306': 754800, '2307': 3300, '2308': 816300, '2309': 300300, '2310': 287700,
    '2311': 486300, '2312': 508200, '2313': 4500, '2314': 14400, '2315': 472800,
    '2316': 371100, '2317': 11700, '2318': 743400, '2319': 444300, '2320': 3900,
    '2321': 9600, '2322': 378600, '2323': 6000, '2324': 845100, '2325': 6900,
    '2326': 1049400, '2327': 752700, '2328': 645600, '2329': 6000, '2330': 9300,
    '2331': 598500, '2332': 7200, '2333': 6600, '2334': 484500, '2335': 636900,
    '2339': 12600, '2340': 570300, '2341': 528000, '2342': 696300, '2343': 556800,
    '2344': 559200, '2345': 456900, '2346': 12300, '2347': 45000, '2348': 140700,
    '2349': 3000, '2350': 424500, '2351': 9600
}

# 3. Frete unitário atualizado
frete_novo = {
    str(2301 + i): round(f, 2) for i, f in enumerate([
        0.45, 0.43, 0.39, 0.47, 0.45, 0.50, 0.48, 0.45, 0.47, 0.38, 0.51, 0.77, 0.82, 0.85, 0.81,
        0.82, 0.91, 0.84, 0.50, 0.44, 0.67, 0.52, 0.60, 0.72, 0.58, 0.52, 0.70, 0.45, 0.35, 0.50,
        0.44, 0.51, 0.66, 0.45, 0.48, 0.59, 0.61, 0.60, 0.71, 0.69, 0.72, 0.75, 0.70, 0.55, 0.55,
        0.60, 0.69, 0.68, 0.76, 0.67, 0.70
    ])
}

# 4. Pares válidos (fábricas permitidas por cliente)
pares_validos = set([
    *[(f'3403208', f'{i}') for i in range(2301, 2352) if i not in (2336, 2337, 2338)],
    *[(f'3423909', str(i)) for i in (
        2305, 2308, 2309, 2310, 2311, 2324, 2326, 2327, 2328, 2329, 2330, 2331, 2332,
        2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2347, 2348,
        2349, 2350, 2351)],
    *[(f'3424402', str(i)) for i in (
        2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2319, 2320,
        2321, 2322, 2324, 2326, 2327, 2328, 2329, 2331, 2332, 2333, 2334, 2335, 2336,
        2337, 2338, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 2351)]
])

# 5. Modelo
prob = LpProblem("Transporte_Correto", LpMinimize)
variaveis = LpVariable.dicts("Qtd", list(pares_validos), lowBound=0, cat='Continuous')

# 6. Função objetivo com frete correto
prob += lpSum(variaveis[fab, cli] * frete_novo[cli] for (fab, cli) in pares_validos)

# 7. Restrições
for fab in fabricas:
    prob += lpSum(variaveis[fab, cli] for (f, cli) in pares_validos if f == fab) <= fabricas[fab]
for cli in demandas:
    prob += lpSum(variaveis[fab, cli] for (fab, c) in pares_validos if c == cli) == demandas[cli]

# 8. Solução
prob.solve()

# 9. Impressão do resultado no mesmo formato que o código anterior
print(f"Status: {LpStatus[prob.status]}")
print(f"Custo Total: {int(value(prob.objective)):06d}\n")

print("Cliente | Fábrica   | Quantidade | Frete Unitário | Custo Total")
print("--------|-----------|------------|----------------|-------------")
for (fab, cli), var in variaveis.items():
    if var.varValue > 0:
        qtd = int(round(var.varValue))
        frete = frete_novo[cli]
        custo = int(round(qtd * frete))
        print(f"{cli:<7} | {fab:<9} | {qtd:10d} | {frete:14.2f} | {custo:11d}")

# Create a DataFrame from the optimization results
resultados_otimizacao = []
for (fab, cli), var in variaveis.items():
    if var.varValue > 0:
        resultados_otimizacao.append({
            'Fabrica': fab,
            'Cliente': cli,
            'Quantidade Transportada': int(round(var.varValue))
        })

df_transporte_otimizado = pd.DataFrame(resultados_otimizacao)

# Salvar o DataFrame em CSV
df_transporte_otimizado.to_csv("Transporte_Correto.csv", index=False)



resultados_otimizacao = []

for (fab, cli), var in variaveis.items():
    if var.varValue > 0:
        qtd = int(round(var.varValue))
        # Use frete_novo dictionary to get the unit freight cost
        valor_unitario = frete_novo.get(cli, 0)
        valor_frete = qtd * valor_unitario

        resultados_otimizacao.append({
            'Cliente': cli,
            'Fabrica': fab,
            'Quantidade Transportada': qtd,
            'Valor por Unidade': valor_unitario,
            'Valor do Frete': valor_frete
        })

df_transporte_otimizado = pd.DataFrame(resultados_otimizacao)

# Format 'Valor por Unidade' to 0,00
df_transporte_otimizado['Valor por Unidade'] = df_transporte_otimizado['Valor por Unidade'].apply(lambda x: f"{x:.2f}".replace('.', ','))

# Format 'Valor do Frete' to integer without points
df_transporte_otimizado['Valor do Frete'] = df_transporte_otimizado['Valor do Frete'].apply(lambda x: f"{int(round(x))}")


# Salvar o DataFrame em CSV
df_transporte_otimizado.to_csv("Transporte_Correto.csv", index=False)



###  UTILIZAÇÃO DAS FABRICAS ##########
resumo = []
for fab in fabricas:
    total_enviado = sum(var.varValue for (f, cli), var in variaveis.items() if f == fab)
    capacidade = fabricas[fab]
    percentual = (total_enviado / capacidade) * 100

    # Formatações
    capacidade_fmt = f"{int(round(capacidade)):,}".replace(",", ".")
    produzido_fmt = f"{int(round(total_enviado)):,}".replace(",", ".")
    percentual_fmt = f"{percentual:.2f}".replace(".", ",")

    resumo.append([fab, capacidade_fmt, produzido_fmt, percentual_fmt])

df_fabrica = pd.DataFrame(resumo, columns=['Fábrica', 'Capacidade', 'Produzido', '% Utilizado'])
df_fabrica.to_csv("resumo_fabrica.csv", index=False)


##### TAXA DE UTILIZAÇÃO POR CLIENTE

# Criar DataFrame de capacidade utilizada por cliente a partir dos resultados da otimização
# Create a list of dictionaries from the optimization results
resultados_otimizacao = []
for (fab, cli), var in variaveis.items():
    if var.varValue > 0:
        resultados_otimizacao.append({
            'Fabrica': fab,
            'Cliente': cli,
            'Quantidade Transportada': int(round(var.varValue))
        })

# Create the df_resultado DataFrame
df_resultado = pd.DataFrame(resultados_otimizacao)

# Criar DataFrame de capacidade utilizada por cliente a partir dos resultados da otimização
df_capacidade = df_resultado[['Fabrica', 'Cliente', 'Quantidade Transportada']].copy()

# Garantir que a coluna 'Fabrica' é string para o .map funcionar corretamente
df_capacidade["Fabrica"] = df_capacidade["Fabrica"].astype(str)

# Capacidade das fábricas (redefinido para garantir que esteja disponível)
fabricas = {
    '3403208': 17000000,
    '3423909': 12500000,
    '3424402': 19500000
}


# Calcular o percentual de uso da capacidade da fábrica
# Certifique-se de que a coluna 'Quantidade Transportada' é numérica
df_capacidade['Quantidade Transportada'] = pd.to_numeric(df_capacidade['Quantidade Transportada'], errors='coerce')

df_capacidade["% da Capacidade da Fábrica"] = (
    df_capacidade["Quantidade Transportada"] / df_capacidade["Fabrica"].map(fabricas) * 100
).round(2).astype(str) + "%"

# Exibir resultado
print("Capacidade utilizada por cliente:")
display(df_capacidade.sort_values(by=["Fabrica", "Cliente"]))

# Exportar CSV
df_capacidade.to_csv("capacidade_utilizada_por_cliente.csv", index=False)

print("\nArquivo 'capacidade_utilizada_por_cliente.csv' salvo com sucesso!")









# Salvar o DataFrame modificado em um novo arquivo CSV
df.to_csv("Rotas_formatado.csv", index=False)

Registros inconsistentes: 0
Empty DataFrame
Columns: [Dt.Pedido, Dt.Emissao, Dt.Entrega, Mes.Base, Ano.Exec, CO.Fabrica, CO.Cliente, Incoterm, Veiculo, Qtd/pallets, Qtd.Transp, Moeda, Vlr.Frete, Inconsistencia_Entrega_Antes_Pedido, Inconsistencia_Emissao_Apos_Entrega]
Index: []

DataFrame após remoção: 38970 registros (inconsistências excluídas)

Registros FOB com Vlr.Frete > 0: 2902 encontrados



,Dt.Pedido,Dt.Emissao,Dt.Entrega,Mes.Base,Ano.Exec,CO.Fabrica,CO.Cliente,Incoterm,Veiculo,Qtd/pallets,Qtd.Transp,Moeda,Vlr.Frete,Inconsistencia_Entrega_Antes_Pedido,Inconsistencia_Emissao_Apos_Entrega,Prazo_Entrega,Valor.Por.Unidade,Vlr.Frete.Num
1627,2023-01-07,2023-01-10,2023-01-13,1,2023,3424402,2302,FOB,P24,24,3600,BRL,"1.015,82",False,False,6,"0,28",1015.82
1768,2023-01-13,2023-01-27,2023-01-30,1,2023,3424402,2306,FOB,P24,24,3600,BRL,"1.091,44",False,False,17,"0,30",1091.44
1802,2023-01-12,2023-01-15,2023-01-17,1,2023,3424402,2308,FOB,P24,24,3600,BRL,"1.036,88",False,False,5,"0,29",1036.88
1803,2023-01-01,2023-01-15,2023-01-19,1,2023,3424402,2308,FOB,P24,24,3600,BRL,"1.036,88",False,False,18,"0,29",1036.88
1895,2023-01-02,2023-01-10,2023-01-15,1,2023,3423909,2329,FOB,P24,24,3000,BRL,"1.038,18",False,False,13,"0,35",1038.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39511,2023-12-09,2023-12-18,2023-12-21,12,2023,3423909,2305,FOB,P12,12,1500,BRL,"359,13",False,False,12,"0,24",359.13
39512,2023-12-21,2023-12-25,2023-12-30,12,2023,3423909,2305,FOB,P12,12,1500,BRL,"359,13",False,False,9,"0,24",359.13
39513,2023-12-16,2023-12-28,2023-12-31,12,2023,3423909,2305,FOB,P12,12,1500,BRL,"359,13",False,False,15,"0,24",359.13
39514,2023-11-27,2023-12-01,2023-12-04,12,2023,3423909,2305,FOB,P12,12,1800,BRL,"413,99",False,False,7,"0,23",413.99


Linhas alteradas de FOB para CIF: 2902
Registros P12 com carga acima de 1800: 18
Registros P12 com excesso de carga:


,Veiculo,Qtd.Transp
39105,P12,3600
39118,P12,3600
39122,P12,3600
39123,P12,3600
39124,P12,3600
39125,P12,3600
39136,P12,3600
39137,P12,3600
39147,P12,3600
39148,P12,3600



Registros com veículo atualizado para P24:


,Veiculo,Qtd.Transp
39105,P24,3600
39118,P24,3600
39122,P24,3600
39123,P24,3600
39124,P24,3600
39125,P24,3600
39136,P24,3600
39137,P24,3600
39147,P24,3600
39148,P24,3600



Registros P24 com carga acima de 3600: 0

Registros com outros tipos de veículo: 0
0        10
3        12
4        12
5        12
6        10
         ..
39511    10
39512    10
39513    10
39514    12
39515    12
Name: Pallets_Transportados, Length: 38970, dtype: object
Soma da quantidade transportada por cliente (somente CIF):
    CO.Cliente  Soma.Qtd.CIF
0         2301       1144800
1         2302        822000
2         2303        436800
3         2304        782700
4         2305       2290500
5         2306       1302600
6         2307       1151400
7         2308       1675200
8         2309       2848800
9         2310       1428600
10        2311       3342300
11        2312        508200
12        2313          4500
13        2314         14400
14        2315        472800
15        2316        371100
16        2317         11700
17        2318        743400
18        2319        792000
19        2320         13200
20        2321         12600
21        2322        673800


,Fabrica,Cliente,Quantidade Transportada,% da Capacidade da Fábrica
37,3403208,2305,601200,3.54%
15,3403208,2308,816300,4.8%
11,3403208,2309,300300,1.77%
34,3403208,2310,287700,1.69%
29,3403208,2311,486300,2.86%
26,3403208,2312,508200,2.99%
17,3403208,2313,4500,0.03%
38,3403208,2314,14400,0.08%
0,3403208,2315,472800,2.78%
32,3403208,2316,371100,2.18%



Arquivo 'capacidade_utilizada_por_cliente.csv' salvo com sucesso!


In [ ]:
####resultado otimização

resultados_otimizacao = []

for (fab, cli), var in variaveis.items():
    if var.varValue > 0:
        qtd = int(round(var.varValue))
        # Use frete_novo dictionary to get the unit freight cost
        valor_unitario = frete_novo.get(cli, 0)
        valor_frete = qtd * valor_unitario

        resultados_otimizacao.append({
            'Cliente': cli,
            'Fabrica': fab,
            'Quantidade Transportada': qtd,
            'Valor por Unidade': valor_unitario,
            'Valor do Frete': valor_frete
        })

df_transporte_otimizado = pd.DataFrame(resultados_otimizacao)

# Format 'Valor por Unidade' to 0,00
df_transporte_otimizado['Valor por Unidade'] = df_transporte_otimizado['Valor por Unidade'].apply(lambda x: f"{x:.2f}".replace('.', ','))

# Format 'Valor do Frete' to integer without points
df_transporte_otimizado['Valor do Frete'] = df_transporte_otimizado['Valor do Frete'].apply(lambda x: f"{int(round(x))}")


# Salvar o DataFrame em CSV
df_transporte_otimizado.to_csv("Transporte_Correto.csv", index=False)

In [ ]:
# Converter datas para formato correto DIA MES ANO (caso haja falhas no parse automático)
df['Dt.Pedido'] = pd.to_datetime(df['Dt.Pedido'], format='%d/%m/%y', errors='coerce')
df['Dt.Emissao'] = pd.to_datetime(df['Dt.Emissao'], format='%d/%m/%y', errors='coerce')
df['Dt.Entrega'] = pd.to_datetime(df['Dt.Entrega'], format='%d/%m/%y', errors='coerce')

# Converter datas para formato correto ANO MES DIA(caso haja falhas no parse automático)
df['Dt.Pedido'] = pd.to_datetime(df['Dt.Pedido'], format='%y/%m/%d', errors='coerce')
df['Dt.Emissao'] = pd.to_datetime(df['Dt.Emissao'], format='%y/%m/%d', errors='coerce')
df['Dt.Entrega'] = pd.to_datetime(df['Dt.Entrega'], format='%y/%m/%d', errors='coerce')


In [ ]:
 # Função para verificar se alguma das datas está em 2022
def contem_2022(row):
    return any(
        dt.year == 2022 if pd.notnull(dt) else False
        for dt in [row['Dt.Pedido'], row['Dt.Emissao'], row['Dt.Entrega']]
    )

# Apagar linhas que possuem qualquer data do ano de 2022
df = df[~df.apply(contem_2022, axis=1)]

In [ ]:
# Remover linhas com datas inválidas
df = df.dropna(subset=['Dt.Pedido', 'Dt.Emissao', 'Dt.Entrega'])
parse_dates=['Dt.Pedido', 'Dt.Emissao', 'Dt.Entrega']


In [ ]:
# Criar flags para inconsistências
df['Inconsistencia_Entrega_Antes_Pedido'] = df['Dt.Entrega'] < df['Dt.Pedido']
df['Inconsistencia_Emissao_Apos_Entrega'] = df['Dt.Emissao'] > df['Dt.Entrega']

# Filtrar registros problemáticos
inconsistencias = df[df['Inconsistencia_Entrega_Antes_Pedido'] | df['Inconsistencia_Emissao_Apos_Entrega']]
print(f"Registros inconsistentes: {len(inconsistencias)}")



Registros inconsistentes: 5


In [ ]:
# Mostra todas as linhas com inconsistências
print(inconsistencias)


       Dt.Pedido Dt.Emissao Dt.Entrega  Mes.Base  Ano.Exec  CO.Fabrica  \
39144 2023-12-12 2023-12-17 2023-12-15        12      2023     3423909   
39151 2023-12-04 2023-12-18 2023-12-16        12      2023     3423909   
39152 2023-12-04 2023-12-18 2023-12-17        12      2023     3423909   
39158 2023-12-10 2023-12-21 2023-12-20        12      2023     3423909   
39161 2023-12-11 2023-12-24 2023-12-23        12      2023     3423909   

       CO.Cliente Incoterm Veiculo  Qtd/pallets  Qtd.Transp Moeda Vlr.Frete  \
39144        2332      FOB     P12           12        1500   BRL      0,00   
39151        2332      FOB     P12           12        3600   BRL      0,00   
39152        2332      FOB     P12           12        3600   BRL      0,00   
39158        2332      FOB     P12           12        1500   BRL      0,00   
39161        2332      FOB     P12           12        3600   BRL      0,00   

       Inconsistencia_Entrega_Antes_Pedido  \
39144                             

In [ ]:
# Remove as inconsistências do DataFrame original (sem criar um novo)
df = df[~(df['Inconsistencia_Entrega_Antes_Pedido'] | df['Inconsistencia_Emissao_Apos_Entrega'])].copy()


print(f"\nDataFrame após remoção: {len(df)} registros (inconsistências excluídas)")



DataFrame após remoção: 39511 registros (inconsistências excluídas)


In [ ]:
#### 4 LEADTIME

import matplotlib.pyplot as plt

# Histograma de prazos de entrega

df['Prazo_Entrega'] = (df['Dt.Entrega'] - df['Dt.Pedido']).dt.days


In [ ]:
# Corrigir emissões após a entrega (exemplo simplificado)
df['Dt.Emissao'] = df.apply(
    lambda row: row['Dt.Pedido'] if row['Dt.Emissao'] > row['Dt.Entrega'] else row['Dt.Emissao'],
    axis=1
)


In [ ]:
# 5 Taxa de ocupação do veículo
#  Cada pallet vale 150 unidades (P12 = 1800 unidades, P24 = 3600 unidades)
import numpy as np

df["Taxa.Ocupacao(%)"] = np.where(
    df["Veiculo"] == "P12",
    round((df["Qtd.Transp"] / 1800) * 100, 2),
    np.where(
        df["Veiculo"] == "P24",
        round((df["Qtd.Transp"] / 3600) * 100, 2),
        None
    )
)

# ✅ Formatar como porcentagem com duas casas decimais
df["Taxa.Ocupacao(%)"] = df["Taxa.Ocupacao(%)"].apply(
    lambda x: f"{x:.2f}%" if pd.notnull(x) else None
)

In [ ]:
# Verificar se as cargas para P12 excedem 1800 unidades
p12_excesso = df[(df['Veiculo'] == 'P12') & (df['Qtd.Transp'] > 1800)]

print(f"Registros P12 com carga acima de 1800: {len(p12_excesso)}")
if not p12_excesso.empty:
    print("Registros P12 com excesso de carga:")
    display(p12_excesso[['Veiculo', 'Qtd.Transp']])
    # Alterar 'P12' para 'P24' para os registros com Qtd.Transp > 1800 (assumindo que 3600 é a Qtd.Transp em excesso)
    df.loc[p12_excesso.index, 'Veiculo'] = 'P24'
    print("\nRegistros com veículo atualizado para P24:")
    display(df.loc[p12_excesso.index, ['Veiculo', 'Qtd.Transp']])


# Verificar se as cargas para P24 excedem 3600 unidades
p24_excesso = df[(df['Veiculo'] == 'P24') & (df['Qtd.Transp'] > 3600)]

print(f"\nRegistros P24 com carga acima de 3600: {len(p24_excesso)}")
if not p24_excesso.empty:
    print("Registros P24 com excesso de carga:")
    display(p24_excesso[['Veiculo', 'Qtd.Transp']])

# Verificar se existem outros tipos de veículo na coluna "Veiculo"
outros_veiculos = df[~df['Veiculo'].isin(['P12', 'P24'])]

print(f"\nRegistros com outros tipos de veículo: {len(outros_veiculos)}")
if not outros_veiculos.empty:
    print("Outros tipos de veículo encontrados:")
    display(outros_veiculos['Veiculo'].value_counts())
    print("\nRegistros com outros tipos de veículo:")
    display(outros_veiculos[['Veiculo', 'Qtd.Transp']])

Registros P12 com carga acima de 1800: 0

Registros P24 com carga acima de 3600: 0

Registros com outros tipos de veículo: 0


In [ ]:
df['Pallets_Transportados'] = df['Qtd.Transp'] / 150
# ✅ Formatar "Pallets_Transportados" como string "00" (para exibição)
df['Pallets_Transportados'] = df['Pallets_Transportados'].apply(lambda x: f"{x:.0f}" if pd.notnull(x) else None)
print(df['Pallets_Transportados'])

0        10
3        12
4        12
5        12
6        10
         ..
39511    10
39512    10
39513    10
39514    12
39515    12
Name: Pallets_Transportados, Length: 38970, dtype: object


In [ ]:
import pandas as pd

# Substituir vírgula por ponto e remover pontos de milhar (formato brasileiro)
df['Vlr.Frete.Num'] = (
    df['Vlr.Frete']
    .astype(str)
    .str.replace('.', '', regex=False)    # Remove milhar
    .str.replace(',', '.', regex=False)   # Troca vírgula por ponto
)

# Converte para float (valores inválidos viram NaN)
df['Vlr.Frete.Num'] = pd.to_numeric(df['Vlr.Frete.Num'], errors='coerce')

# Filtrar registros onde Incoterm é FOB e valor de frete > 0
fob_com_frete = df[(df['Incoterm'] == 'FOB') & (df['Vlr.Frete.Num'] > 0)].copy()

# Mostrar resultado
print(f"\nRegistros FOB com Vlr.Frete > 0: {len(fob_com_frete)} encontrados\n")
display(fob_com_frete)



Registros FOB com Vlr.Frete > 0: 2902 encontrados



,Dt.Pedido,Dt.Emissao,Dt.Entrega,Mes.Base,Ano.Exec,CO.Fabrica,CO.Cliente,Incoterm,Veiculo,Qtd/pallets,Qtd.Transp,Moeda,Vlr.Frete,Inconsistencia_Entrega_Antes_Pedido,Inconsistencia_Emissao_Apos_Entrega,Prazo_Entrega,Valor.Por.Unidade,Taxa.Ocupacao(%),Vlr.Frete.Num
1627,2023-01-07,2023-01-10,2023-01-13,1,2023,3424402,2302,FOB,P24,24,3600,BRL,"1.015,82",False,False,6,"0,28",100.00%,1015.82
1768,2023-01-13,2023-01-27,2023-01-30,1,2023,3424402,2306,FOB,P24,24,3600,BRL,"1.091,44",False,False,17,"0,30",100.00%,1091.44
1802,2023-01-12,2023-01-15,2023-01-17,1,2023,3424402,2308,FOB,P24,24,3600,BRL,"1.036,88",False,False,5,"0,29",100.00%,1036.88
1803,2023-01-01,2023-01-15,2023-01-19,1,2023,3424402,2308,FOB,P24,24,3600,BRL,"1.036,88",False,False,18,"0,29",100.00%,1036.88
1895,2023-01-02,2023-01-10,2023-01-15,1,2023,3423909,2329,FOB,P24,24,3000,BRL,"1.038,18",False,False,13,"0,35",83.33%,1038.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39511,2023-12-09,2023-12-18,2023-12-21,12,2023,3423909,2305,FOB,P12,12,1500,BRL,"359,13",False,False,12,"0,24",83.33%,359.13
39512,2023-12-21,2023-12-25,2023-12-30,12,2023,3423909,2305,FOB,P12,12,1500,BRL,"359,13",False,False,9,"0,24",83.33%,359.13
39513,2023-12-16,2023-12-28,2023-12-31,12,2023,3423909,2305,FOB,P12,12,1500,BRL,"359,13",False,False,15,"0,24",83.33%,359.13
39514,2023-11-27,2023-12-01,2023-12-04,12,2023,3423909,2305,FOB,P12,12,1800,BRL,"413,99",False,False,7,"0,23",100.00%,413.99


In [ ]:
import pandas as pd

# Corrigir valor de frete (formato brasileiro) para float
df['Vlr.Frete.Num'] = (
    df['Vlr.Frete']
    .astype(str)
    .str.replace('.', '', regex=False)    # Remove milhar
    .str.replace(',', '.', regex=False)   # Troca vírgula por ponto decimal
)

df['Vlr.Frete.Num'] = pd.to_numeric(df['Vlr.Frete.Num'], errors='coerce')

# Criar condição
condicao_fob_com_frete = (df['Incoterm'] == 'FOB') & (df['Vlr.Frete.Num'] > 0)

# Aplicar a substituição
df.loc[condicao_fob_com_frete, 'Incoterm'] = 'CIF'

# (Opcional) Exibir quantas linhas foram alteradas
print(f"Linhas alteradas de FOB para CIF: {condicao_fob_com_frete.sum()}")


Linhas alteradas de FOB para CIF: 2902


In [ ]:
import pandas as pd

# Corrigir valor de frete para float
df['Vlr.Frete.Num'] = (
    df['Vlr.Frete']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)
df['Vlr.Frete.Num'] = pd.to_numeric(df['Vlr.Frete.Num'], errors='coerce')
df['Qtd.Transp'] = pd.to_numeric(df['Qtd.Transp'], errors='coerce')

# Remover valores nulos ou inválidos
df_valid = df[(df['Qtd.Transp'] > 0) & df['Vlr.Frete.Num'].notnull()]

# Calcular valor unitário por linha
df_valid['Vlr.Unitario'] = df_valid['Vlr.Frete.Num'] / df_valid['Qtd.Transp']

# Calcular média por cliente
media_por_cliente = df_valid.groupby('CO.Cliente')['Vlr.Unitario'].mean().reset_index()

# Formatar a média no padrão 0,00 (brasileiro)
media_por_cliente['Vlr.Unitario'] = media_por_cliente['Vlr.Unitario'].apply(
    lambda x: f"{x:,.2f}".replace('.', 'v').replace(',', '.').replace('v', ',')
)

# Exibir resultado
print("Média do valor por unidade transportada por cliente (formato 0,00):")
print(media_por_cliente)

# Salvar resultado formatado em CSV
media_por_cliente.to_csv('media_valor_unitario_por_cliente.csv', index=False, encoding='utf-8-sig')

print("\nArquivo 'media_valor_unitario_por_cliente.csv' salvo com sucesso!")



Média do valor por unidade transportada por cliente (formato 0,00):
    CO.Cliente Vlr.Unitario
0         2301         0,21
1         2302         0,16
2         2303         0,10
3         2304         0,43
4         2305         0,41
5         2306         0,45
6         2307         0,26
7         2308         0,41
8         2309         0,46
9         2310         0,37
10        2311         0,48
11        2312         0,72
12        2313         0,01
13        2314         0,02
14        2315         0,73
15        2316         0,74
16        2317         0,02
17        2318         0,79
18        2319         0,44
19        2320         0,00
20        2321         0,01
21        2322         0,45
22        2323         0,00
23        2324         0,25
24        2325         0,00
25        2326         0,47
26        2327         0,22
27        2328         0,42
28        2329         0,10
29        2330         0,14
30        2331         0,38
31        2332         0,00
32      

In [ ]:
import pandas as pd

# Corrigir valor de frete (formato brasileiro) para float
df['Vlr.Frete.Num'] = (
    df['Vlr.Frete']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)
df['Vlr.Frete.Num'] = pd.to_numeric(df['Vlr.Frete.Num'], errors='coerce')
df['Qtd.Transp'] = pd.to_numeric(df['Qtd.Transp'], errors='coerce')

# Filtrar somente linhas com Incoterm = CIF
df_cif = df[(df['Incoterm'] == 'CIF') & (df['Qtd.Transp'] > 0) & df['Vlr.Frete.Num'].notnull()].copy()

# Calcular valor unitário por linha
df_cif['Vlr.Unitario'] = df_cif['Vlr.Frete.Num'] / df_cif['Qtd.Transp']

# Calcular média por cliente
media_por_cliente = df_cif.groupby('CO.Cliente')['Vlr.Unitario'].mean().reset_index()

# Formatar a média no padrão 0,00 (brasileiro)
media_por_cliente['Vlr.Unitario'] = media_por_cliente['Vlr.Unitario'].apply(
    lambda x: f"{x:,.2f}".replace('.', 'v').replace(',', '.').replace('v', ',')
)

# Salvar resultado em CSV
media_por_cliente.to_csv('media_valor_unitario_CIF_por_cliente.csv', index=False, encoding='utf-8-sig')
print("\n✅ Arquivo 'media_valor_unitario_CIF_por_cliente.csv' salvo com sucesso!")


print(media_por_cliente)


✅ Arquivo 'media_valor_unitario_CIF_por_cliente.csv' salvo com sucesso!
    CO.Cliente Vlr.Unitario
0         2301         0,45
1         2302         0,43
2         2303         0,39
3         2304         0,47
4         2305         0,45
5         2306         0,50
6         2307         0,48
7         2308         0,45
8         2309         0,47
9         2310         0,38
10        2311         0,51
11        2312         0,77
12        2313         0,82
13        2314         0,85
14        2315         0,81
15        2316         0,82
16        2317         0,91
17        2318         0,84
18        2319         0,50
19        2320         0,44
20        2321         0,67
21        2322         0,52
22        2323         0,60
23        2324         0,72
24        2325         0,58
25        2326         0,52
26        2327         0,70
27        2328         0,45
28        2329         0,35
29        2330         0,50
30        2331         0,44
31        2332         0,51
32 

In [ ]:
import pandas as pd

# Garantir tipo numérico
df['Qtd.Transp'] = pd.to_numeric(df['Qtd.Transp'], errors='coerce')

# Filtrar apenas linhas com Incoterm = CIF
df_cif = df[df['Incoterm'] == 'CIF'].copy()

# Agrupar por cliente e somar
soma_cif_por_cliente = df_cif.groupby('CO.Cliente')['Qtd.Transp'].sum().reset_index()

# Renomear coluna para clareza
soma_cif_por_cliente.rename(columns={'Qtd.Transp': 'Soma.Qtd.CIF'}, inplace=True)

# Exibir resultado
print("Soma da quantidade transportada por cliente (somente CIF):")
print(soma_cif_por_cliente)


Soma da quantidade transportada por cliente (somente CIF):
    CO.Cliente  Soma.Qtd.CIF
0         2301       1144800
1         2302        822000
2         2303        436800
3         2304        782700
4         2305       2290500
5         2306       1302600
6         2307       1151400
7         2308       1675200
8         2309       2848800
9         2310       1428600
10        2311       3342300
11        2312        508200
12        2313          4500
13        2314         14400
14        2315        472800
15        2316        371100
16        2317         11700
17        2318        743400
18        2319        792000
19        2320         13200
20        2321         12600
21        2322        673800
22        2323          9000
23        2324        849900
24        2325          9900
25        2326       1927800
26        2327        768600
27        2328       2155500
28        2329        602400
29        2330        594000
30        2331       1867500
31        233

In [ ]:
soma_cif_por_cliente.to_csv('soma_qtd_transportada_CIF_por_cliente.csv', index=False, encoding='utf-8-sig')

print("\nArquivo 'soma_qtd_transportada_CIF_por_cliente.csv' salvo com sucesso!")



Arquivo 'soma_qtd_transportada_CIF_por_cliente.csv' salvo com sucesso!


In [ ]:
clientes_por_fabrica = df.groupby('CO.Fabrica')['CO.Cliente'].unique().reset_index()

# (Opcional) transformar listas em string separada por vírgulas
clientes_por_fabrica['Clientes'] = clientes_por_fabrica['CO.Cliente'].apply(lambda x: ', '.join(map(str, sorted(set(x)))))

# Mostrar resultado
print("Clientes únicos por fábrica:")
print(clientes_por_fabrica[['CO.Fabrica','Clientes' ]])

clientes_por_fabrica[['CO.Fabrica', ]].to_csv('clientes_por_fabrica.csv', index=False, encoding='utf-8-sig')
print("\nArquivo 'clientes_por_fabrica.csv' salvo com sucesso!")

clientes_por_fabrica.to_csv('clientes_por_fabrica.csv', index=False, encoding='utf-8-sig')
print("\nArquivo 'clientes_por_fabrica.csv' salvo com sucesso!")

Clientes únicos por fábrica:
   CO.Fabrica                                           Clientes
0     3403208  2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308...
1     3423909  2305, 2308, 2309, 2310, 2311, 2324, 2325, 2326...
2     3424402  2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308...

Arquivo 'clientes_por_fabrica.csv' salvo com sucesso!

Arquivo 'clientes_por_fabrica.csv' salvo com sucesso!


In [ ]:
import pandas as pd

# Agrupar e obter clientes únicos
clientes_por_fabrica = df.groupby('CO.Fabrica')['CO.Cliente'].unique().reset_index()

# Transformar listas de clientes em strings separadas por vírgula
clientes_por_fabrica['CO.Cliente'] = clientes_por_fabrica['CO.Cliente'].apply(lambda x: ','.join(map(str, x)))

# Salvar como CSV separado por vírgula
clientes_por_fabrica.to_csv('clientes_por_fabrica.csv', index=False)


In [ ]:
########################### OTIMIZAÇÃO CIF ############################################




!pip install pulp
from pulp import *
import pandas as pd

# 1. Capacidade das fábricas
fabricas = {
    '3403208': 17000000,
    '3423909': 12500000,
    '3424402': 19500000
}

# 2. Demandas dos clientes
demandas = {
    '2301': 7800, '2302': 5400, '2303': 5400, '2304': 404700, '2305': 601200,
    '2306': 754800, '2307': 3300, '2308': 816300, '2309': 300300, '2310': 287700,
    '2311': 486300, '2312': 508200, '2313': 4500, '2314': 14400, '2315': 472800,
    '2316': 371100, '2317': 11700, '2318': 743400, '2319': 444300, '2320': 3900,
    '2321': 9600, '2322': 378600, '2323': 6000, '2324': 845100, '2325': 6900,
    '2326': 1049400, '2327': 752700, '2328': 645600, '2329': 6000, '2330': 9300,
    '2331': 598500, '2332': 7200, '2333': 6600, '2334': 484500, '2335': 636900,
    '2339': 12600, '2340': 570300, '2341': 528000, '2342': 696300, '2343': 556800,
    '2344': 559200, '2345': 456900, '2346': 12300, '2347': 45000, '2348': 140700,
    '2349': 3000, '2350': 424500, '2351': 9600
}

# 3. Frete unitário atualizado
frete_novo = {
    str(2301 + i): round(f, 2) for i, f in enumerate([
        0.45, 0.43, 0.39, 0.47, 0.45, 0.50, 0.48, 0.45, 0.47, 0.38, 0.51, 0.77, 0.82, 0.85, 0.81,
        0.82, 0.91, 0.84, 0.50, 0.44, 0.67, 0.52, 0.60, 0.72, 0.58, 0.52, 0.70, 0.45, 0.35, 0.50,
        0.44, 0.51, 0.66, 0.45, 0.48, 0.59, 0.61, 0.60, 0.71, 0.69, 0.72, 0.75, 0.70, 0.55, 0.55,
        0.60, 0.69, 0.68, 0.76, 0.67, 0.70
    ])
}

# 4. Pares válidos (fábricas permitidas por cliente)
pares_validos = set([
    *[(f'3403208', f'{i}') for i in range(2301, 2352) if i not in (2336, 2337, 2338)],
    *[(f'3423909', str(i)) for i in (
        2305, 2308, 2309, 2310, 2311, 2324, 2326, 2327, 2328, 2329, 2330, 2331, 2332,
        2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2347, 2348,
        2349, 2350, 2351)],
    *[(f'3424402', str(i)) for i in (
        2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2319, 2320,
        2321, 2322, 2324, 2326, 2327, 2328, 2329, 2331, 2332, 2333, 2334, 2335, 2336,
        2337, 2338, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 2351)]
])

# 5. Modelo
prob = LpProblem("Transporte_Correto", LpMinimize)
variaveis = LpVariable.dicts("Qtd", list(pares_validos), lowBound=0, cat='Continuous')

# 6. Função objetivo com frete correto
prob += lpSum(variaveis[fab, cli] * frete_novo[cli] for (fab, cli) in pares_validos)

# 7. Restrições
for fab in fabricas:
    prob += lpSum(variaveis[fab, cli] for (f, cli) in pares_validos if f == fab) <= fabricas[fab]
for cli in demandas:
    prob += lpSum(variaveis[fab, cli] for (fab, c) in pares_validos if c == cli) == demandas[cli]

# 8. Solução
prob.solve()

# 9. Impressão do resultado no mesmo formato que o código anterior
print(f"Status: {LpStatus[prob.status]}")
print(f"Custo Total: {int(value(prob.objective)):06d}\n")

print("Cliente | Fábrica   | Quantidade | Frete Unitário | Custo Total")
print("--------|-----------|------------|----------------|-------------")
for (fab, cli), var in variaveis.items():
    if var.varValue > 0:
        qtd = int(round(var.varValue))
        frete = frete_novo[cli]
        custo = int(round(qtd * frete))
        print(f"{cli:<7} | {fab:<9} | {qtd:10d} | {frete:14.2f} | {custo:11d}")


Status: Optimal
Custo Total: 9332274

Cliente | Fábrica   | Quantidade | Frete Unitário | Custo Total
--------|-----------|------------|----------------|-------------
2315    | 3403208   |     472800 |           0.81 |      382968
2330    | 3423909   |       9300 |           0.50 |        4650
2304    | 3424402   |     404700 |           0.47 |      190209
2306    | 3424402   |     754800 |           0.50 |      377400
2326    | 3403208   |    1049400 |           0.52 |      545688
2324    | 3403208   |     845100 |           0.72 |      608472
2332    | 3403208   |       7200 |           0.51 |        3672
2303    | 3424402   |       5400 |           0.39 |        2106
2323    | 3403208   |       6000 |           0.60 |        3600
2318    | 3403208   |     743400 |           0.84 |      624456
2345    | 3424402   |     456900 |           0.55 |      251295
2309    | 3403208   |     300300 |           0.47 |      141141
2334    | 3403208   |     484500 |           0.45 |      218025
2

In [ ]:
resumo = []
for fab in fabricas:
    total_enviado = sum(var.varValue for (f, cli), var in variaveis.items() if f == fab)
    capacidade = fabricas[fab]
    percentual = (total_enviado / capacidade) * 100

    # Formatações
    capacidade_fmt = f"{int(round(capacidade)):,}".replace(",", ".")
    produzido_fmt = f"{int(round(total_enviado)):,}".replace(",", ".")
    percentual_fmt = f"{percentual:.2f}".replace(".", ",")

    resumo.append([fab, capacidade_fmt, produzido_fmt, percentual_fmt])

df_fabrica = pd.DataFrame(resumo, columns=['Fábrica', 'Capacidade', 'Produzido', '% Utilizado'])
df_fabrica.to_csv("resumo_fabrica.csv", index=False)

In [ ]:
# 📈 10. Soma de Quantidade Transportada por Cliente
soma_transp_cliente = df.groupby("CO.Cliente")["Qtd.Transp"].sum().reset_index(name="Soma.Qtd.Transp")
soma_transp_cliente.to_csv("Soma_Transp_Cliente.csv", index=False)

In [ ]:
# 🏭 11. Lista de clientes únicos por fábrica
clientes_por_fabrica = df.groupby("CO.Fabrica")["CO.Cliente"].unique().reset_index(name="Clientes")
clientes_por_fabrica.to_csv("Clientes_Por_Fabrica.csv", index=False)

In [ ]:
# Criar DataFrame de capacidade utilizada por cliente a partir dos resultados da otimização
# Create a list of dictionaries from the optimization results
resultados_otimizacao = []
for (fab, cli), var in variaveis.items():
    if var.varValue > 0:
        resultados_otimizacao.append({
            'Fabrica': fab,
            'Cliente': cli,
            'Quantidade Transportada': int(round(var.varValue))
        })

# Create the df_resultado DataFrame
df_resultado = pd.DataFrame(resultados_otimizacao)

# Criar DataFrame de capacidade utilizada por cliente a partir dos resultados da otimização
df_capacidade = df_resultado[['Fabrica', 'Cliente', 'Quantidade Transportada']].copy()

# Garantir que a coluna 'Fabrica' é string para o .map funcionar corretamente
df_capacidade["Fabrica"] = df_capacidade["Fabrica"].astype(str)

# Capacidade das fábricas (redefinido para garantir que esteja disponível)
fabricas = {
    '3403208': 17000000,
    '3423909': 12500000,
    '3424402': 19500000
}


# Calcular o percentual de uso da capacidade da fábrica
# Certifique-se de que a coluna 'Quantidade Transportada' é numérica
df_capacidade['Quantidade Transportada'] = pd.to_numeric(df_capacidade['Quantidade Transportada'], errors='coerce')

df_capacidade["% da Capacidade da Fábrica"] = (
    df_capacidade["Quantidade Transportada"] / df_capacidade["Fabrica"].map(fabricas) * 100
).round(2).astype(str) + "%"

# Exibir resultado
print("Capacidade utilizada por cliente:")
display(df_capacidade.sort_values(by=["Fabrica", "Cliente"]))

# Exportar CSV
df_capacidade.to_csv("capacidade_utilizada_por_cliente.csv", index=False)

print("\nArquivo 'capacidade_utilizada_por_cliente.csv' salvo com sucesso!")

Capacidade utilizada por cliente:


,Fabrica,Cliente,Quantidade Transportada,% da Capacidade da Fábrica
6,3403208,2305,601200,3.54%
11,3403208,2308,816300,4.8%
44,3403208,2309,300300,1.77%
43,3403208,2310,287700,1.69%
17,3403208,2311,486300,2.86%
45,3403208,2312,508200,2.99%
23,3403208,2313,4500,0.03%
15,3403208,2314,14400,0.08%
27,3403208,2315,472800,2.78%
20,3403208,2316,371100,2.18%



Arquivo 'capacidade_utilizada_por_cliente.csv' salvo com sucesso!
